In [1]:
import cv2
import mediapipe as mp
import math
import time
import numpy as np
# Initialisation de MediaPipe Pose.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()


In [2]:
def calculate_angle(landmark1, landmark2, landmark3):
    # Calculer l'angle formé par trois points de repère
    a = math.sqrt((landmark2["x"] - landmark1["x"]) ** 2 + (landmark2["y"] - landmark1["y"]) ** 2)
    b = math.sqrt((landmark2["x"] - landmark3["x"]) ** 2 + (landmark2["y"] - landmark3["y"]) ** 2)
    c = math.sqrt((landmark3["x"] - landmark1["x"]) ** 2 + (landmark3["y"] - landmark1["y"]) ** 2)
    angle = math.acos((a ** 2 + b ** 2 - c ** 2) / (2 * a * b))
    return math.degrees(angle)

In [3]:


def capture_mouvement(a1,a2,a3):
    cap = cv2.VideoCapture(0)
    
    
    list_angle = []
    start_time = time.time()

    while cap.isOpened():
        text = ''
        p2, p1, p3 = None, None, None
        
        ret, frame = cap.read()
        if not ret:
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            for id, lm in enumerate(results.pose_landmarks.landmark):
                height, width, _ = frame.shape
                x, y = int(lm.x * width), int(lm.y * height)
                cv2.putText(frame, str(id), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                if lm.visibility >= 0.90:
                    
                    if id == a1: 
                        p1 = lm
                    elif id == a2:  
                        p2 = lm
                    elif id == a3:  
                        p3 = lm

            if p2 and p1 and p3:
                current_time = time.time()
                timing = current_time - start_time
                text = str(timing)
                #Calcul Angle
                angle = calculate_angle(p1, p2, p3)
                angle_text = str(int(angle))  # Arrondi à l'entier le plus proche et ajout du symbole degré

                #Affichage 
                x = (p1.x + p3.x) / 2
                y = (p1.y + p3.y) / 2

                x, y = int(x * width), int(y * height)

                cv2.putText(frame, angle_text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (102, 0, 255), 2)
                    
                
                if timing >= 5:  # Capture l'angle toutes les 5 secondes
                    
                    list_angle.append(angle)
                
                    start_time = current_time

        font = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (10, 400)
        fontScale = 1
        fontColor = (102, 0, 255)
        lineType = 2
        cv2.putText(frame, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Pose Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    return list_angle




In [4]:
# list = capture_mouvement(11,13,15)

In [5]:
print(list)

<class 'list'>


### 1er Exo (dev militaire)

- a 1 ) 11-13-15
- a 2 ) 12-14-16
    
        Init/ 60-64

        End/ 145-150

- b 1 ) 13-11-23
- b 2 ) 14-12-24

        Init/ 80-90

        End/ 155-165

- c 1 ) 14.y == 13.y
- c 2 ) 16.y == 12.y

- E 0 ) 7.y == 21.y ; 8.y == 22.y
- E 1 ) 15.y == ( dist(11,13) + dist(13,15)) + 11.y  ;  16.y == ( dist(12,14) + dist(14,16)) + 12.y


In [6]:
def is_angled(cdt,dict_id):
    
    
    if(cdt == "a1") :
        target = 80
        a1 = dict_id[11]
        
        a2 = dict_id[13]
        a3 = dict_id[15]
        
    if(cdt == "a2") :
        target = 80
        a1 = dict_id[12]
        a2 = dict_id[14]
        a3 = dict_id[16]
        
        
    return abs(target - calculate_angle(a1,a2,a3)) <= 5 ,  calculate_angle(a1,a2,a3)
    
    

#### E0

In [7]:
def E0(dict_id):
    if dict_id[7] and dict_id[8] and dict_id[21] and dict_id[22]:
        
        return abs(dict_id[7]["y"] - dict_id[21]["y"]) <= 60 and abs(dict_id[8]["y"] - dict_id[22]["y"]) <= 60
    else :
        return False
        
    

#### E1

In [8]:
def calculate_dist(lm1, lm2, lm3):
    # Calcul de la distance entre lm1 et lm2
    dist1 = math.sqrt((lm2["x"] - lm1["x"])**2 + (lm2["y"] - lm1["y"])**2)

    # Calcul de la distance entre lm2 et lm3
    dist2 = math.sqrt((lm3["x"] - lm2["x"])**2 + (lm3["y"] - lm2["y"])**2)

    # Addition des deux distances
    total_dist = dist1 + dist2

    return total_dist

In [9]:
def E1(dict_id,lenght1, lenght2):
    if dict_id[15] and dict_id[16]  :
        # print(dict_id[15]["y"])
        # print(lenght1)
        
        return abs(dict_id[15]["y"] - lenght1) <= 10 and abs(dict_id[16]["y"] - lenght2) <= 10

In [10]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def calibrate():
    dict_id = {
        11: None,
        13: None,
        15: None,
    }

    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        height, width, _ = frame.shape
        if not ret:
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            for id, lm in enumerate(results.pose_landmarks.landmark):
                
                x, y = int(lm.x * width), int(lm.y * height)

                if lm.visibility >= 0.90:
                    dict_id[id] = {'x': x, 'y': y}
                    
        cv2.putText(frame, "Levez les bras", (int(width/2 -100), 20 ), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("Calibration", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or (dict_id[15] and dict_id[13] and dict_id[11]):
            break

    cap.release()
    cv2.destroyAllWindows()

    #return int( - ( dict_id[11]["y"] - calculate_dist(dict_id[11], dict_id[13], dict_id[15]) ) ), int(dict_id[11]["y"] ) , int(dict_id[12]["y"] )
    return int( calculate_dist(dict_id[11], dict_id[13], dict_id[15]) )

In [11]:
# calibrate()

In [12]:
from transitions import Machine

class StateMachine:
    states = ['A', 'B', 'C']

    def __init__(self):
        self.machine = Machine(model=self, states=StateMachine.states, initial='A')
        self.machine.add_transition(trigger='to_B', source='A', dest='B')
        self.machine.add_transition(trigger='to_A', source='B', dest='A')
        self.machine.add_transition(trigger='to_C', source='B', dest='C')


A : init
B : first level
C : second level  (+1)

In [13]:


def track(nbRep):
    cap = cv2.VideoCapture(0)
    
    nb = 0
    text = f'0 / {nbRep}'
    dict_id = {
        7 : None,
        8 : None,
        21 : None,
        22 : None,
        11 : None,
        13 : None,
        15 : None,
        12 : None,
        14 : None,
        16 : None,
    }
    
    machine = StateMachine()
    
    lenght = None
   

    while cap.isOpened():
        
       
        
        # Couleur du cercle en BGR (Bleu, Vert, Rouge)
        colorA1,colorA2 = (255, 0, 0) , (255, 0, 0) # Un cercle bleu
        ret, frame = cap.read()
       
        if not ret:
            continue
        
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            for id, lm in enumerate(results.pose_landmarks.landmark):
                height, width, _ = frame.shape
                x, y = int(lm.x * width), int(lm.y * height)
                cv2.putText(frame, str(id), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                if lm.visibility >= 0.90:
                    p = {
                        'x' : None,
                        'y' : None,
                    }
                    p['x'] = x
                    p['y'] = y
                    dict_id[id] = p
                    
            
            #Update the lenght only on state A and C (so it don't change during state B)
            if machine.state == "A" or machine.state == "C":
                if dict_id[11] and dict_id[12] and dict_id[13] and dict_id[15]  :
            
                    lenght = int( calculate_dist(dict_id[11], dict_id[13], dict_id[15]) )
                    
                    
                    
                        
            if dict_id[11] and dict_id[12] and lenght != None :
                #Bottom Line
                y11 = int(dict_id[11]["y"])
                y12 = int(dict_id[12]["y"])
                
                cv2.line(frame,(0, y11  ),(width, y12 ),(255,0,0),2)
                
                #Top Line
                #lenght = lenght of the arm
                cv2.line(frame,(0,  -(lenght - y12) + 15 ),(width, -( lenght - y11) +15),(255,0,0),2)
            
            
            
            #If elbow in lower bound 
            if E0(dict_id) and machine.state == "A" or machine.state == "C":
                
                #Correct angle
                
                y11 = int(dict_id[11]["y"])
                x11 = int(dict_id[11]["x"])
                
                y13 = int(dict_id[13]["y"])
                x13 = int(dict_id[13]["x"])
                
                y14 = int(dict_id[14]["y"])
                x14 = int(dict_id[14]["x"])
                
                y12 = int(dict_id[12]["y"])
                x12 = int(dict_id[12]["x"])
                
                angle_1113 = math.atan2(y13 - y11, x13 - x11)
                # Convertir alpha en radians et ajuster pour l'inversion de l'axe y
                alpha_rad = 80
                # Calculer le nouvel angle
                new_angle = angle_1113 + alpha_rad

                # Calculer les nouvelles coordonnées du point C à une distance length de A
                Cx = int( x13 - 500 * math.cos(new_angle * 3.14 /180) )
                Cy = int( y13 - 500 * math.sin(new_angle *3.14 /180)  )# Soustraction due à l'inversion de l'axe y

               

                angle_1214 = math.atan2(y14 - y12, x14 - x12)
                
                # Calculer le nouvel angle
                new_angle_2 =  -(angle_1214 + alpha_rad)
                
                xa2 = int( x14 + 500 * math.cos((new_angle_2 ) * 3.14 / 180))
                ya2 = int( y14 + 500 * math.sin((new_angle_2 ) * 3.14 /180))

                
                
                #A1
                a1 , angle1 = is_angled("a1",dict_id)
                
                if a1 :
                    colorA1 = (0, 255, 0)  # Un cercle vert
                   
                else :
                    colorA1 = (0,0,255)
                    
                #A2
                a2, angle2 = is_angled("a2",dict_id)
                
                if a2 :
                    colorA2 =(0, 255, 0)
                    
                else :
                    colorA2 = (0,0,255)
                
                if a1 and a2 :
                    machine.to_B()
                
                cv2.line(frame,(x13, y13  ),(Cx, Cy ),colorA1,2)
                cv2.line(frame,(x14, y14  ),(xa2, ya2 ),colorA2,2)
            
            
            
            #If hands on upper bound
            if lenght != None :
                if E1(dict_id, -(lenght - y11) + 15 , -(lenght - y12) + 15 ) and machine.state == "B" :
                    
                    machine.to_C()
                    nb += 1
                    text = f"{nb} / {nbRep}"
                    
              
            
    
        #Circles
        if dict_id[13] :
            cv2.circle(frame, (dict_id[13]["x"],dict_id[13]["y"]), 20, colorA1, 2)
        if dict_id[14] :
            cv2.circle(frame, (dict_id[14]["x"],dict_id[14]["y"]), 20, colorA2, 2)
            
        font = cv2.FONT_HERSHEY_SIMPLEX
        bottomLeftCornerOfText = (10, 400)
        fontScale = 1
        fontColor = (102, 0, 255)
        lineType = 2
        
        
        
              
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        frame = cv2.flip(frame,1)
        cv2.putText(frame, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
        
        
        if nb >= nbRep :
            break

        cv2.imshow("Pose Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    




In [14]:
def start():
    # lenght = None
    # while lenght == None :
    #     lenght=  calibrate() 
    
    track(10)
    